In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
import urllib.parse

report_date = datetime.now()

# end = datetime.strptime(report_date, '%m/%d/%Y')
start = date(report_date.year - 2, 1, 1)

donatedFoodValue = 127206.12

import config
base_url = config.church_domain_url + r"/events/reports/243292484#/"

import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(report_date.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292484#/start_date=01%2F01%2F2022&end_date=06%2F03%2F2024&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-06-01-2024 (3).xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-05-31-2024.xlsx"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)

# drop the Total row
attendance = attendance[attendance['Breeze ID'].notna()]

people = pd.read_excel(peopleexport)
print('{count} people in input.'.format(count = len(attendance.index)))

461 people in input.


In [3]:
# Keep only the columns we use.

attendance = attendance.drop([
    'First Name', 
    'Last Name', 
    'Person Totals'
], axis = 1)

dates = attendance.columns[attendance.columns != 'Breeze ID']

family_size = {k: v for k, v in zip(people['Breeze ID'], people['How many people live in your household? (including yourself)'])}
seniors = {k: v for k, v in zip(people['Breeze ID'], people['Total number of people aged 60 or older in Household?'])}


In [12]:
import math

attendance.fillna(0, inplace=True)

numberOfSeniors = 25

people_fed = {}
seniors_fed = {}
for date in dates:
    datesum = 0
    seniorcount = 0
    for _, row in attendance.iterrows():
        if row[date] == 'X':
            datesum += family_size[row['Breeze ID']] if not math.isnan(family_size[row['Breeze ID']]) else 0
            seniorcount += seniors[row['Breeze ID']] if not math.isnan(seniors[row['Breeze ID']]) else 0
    people_fed[date] = datesum + numberOfSeniors if datesum > 0 else 0
    seniors_fed[date] = seniorcount + numberOfSeniors if datesum > 0 else 0



In [13]:
totalFed = sum(people_fed.values())

output = pd.DataFrame({'Date': dates, 
                       'People Fed': people_fed.values(), 
                       'Seniors Fed': seniors_fed.values(),
                       'Value': [round(p * donatedFoodValue / totalFed, 2) for p in people_fed.values()] })

In [14]:
people_fed_file = r"C:\Users\ralph\Downloads\People Fed.csv"
output.to_csv(people_fed_file, index=False)